# View shape changes in PCA

## Setup

In [2]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)
print("Directory added to path: ", sys_dir)

Working directory:  /home/nmiolane/code/my28brains
Directory added to path:  /home/nmiolane/code


## Imports

In [3]:
import glob

import numpy as np
import trimesh
from sklearn.decomposition import PCA

CENTERED_MESHES_DIR = os.path.join(os.getcwd(), "data", "registered_meshes")
print("CENTERED_MESHES_DIR: ", CENTERED_MESHES_DIR)

CENTERED_MESHES_DIR:  /home/nmiolane/code/my28brains/data/registered_meshes


## Reparameterize

Currently, each mesh of the time-series has a different number of vertices and even if they had the same nnumber of vertices, these vertices would not correspond across meshes.

We say that the surface mesh are "unparameterized" in the sense that they are not parameterized in the same way.

We would like to use geomstats code to perform statistics on these surface meshes, however geomstats code only posssesses the parameterized version for now. 

Thus, we will reparameterize the curves so that they have the same number of vertices, and matching vertices and faces.

To this aim:
- We consider the mesh parameterization of the first mesh of each time-series: i.e., the. base mesh `B`.
- We will register the base mesh `B` to each of the other mesh `M` and get a transformation `phi`.
- We will apply the ransformation `phi` to the base mesh `B`.


--> This will give a new mesh `phi * B` that looks like M but is parameterized like B.

In [4]:
hemisphere = "left"
structure_id = -1
string_base = os.path.join(
    CENTERED_MESHES_DIR, f"{hemisphere}_structure_{structure_id}_sub-01_ses-**.ply"
)
paths = sorted(glob.glob(string_base))

print(
    f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:\n {paths}\n"
)

Found 27 ply files for left hemisphere and anatomical structure -1:
 ['/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-02.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-03.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-04.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-05.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-06.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-07.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-08.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-09.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-10.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-11.ply', 

In [6]:
base_path = "/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-02.ply"
one_path = "/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-03.ply"

base_mesh = trimesh.load(base_path)
one_mesh = trimesh.load(one_path)

In [13]:
reparameterized = []
for path in paths:
    print(f"\tLoad mesh from path: {path}")
    mesh = trimesh.load(path)
    vertices = np.array(mesh.vertices)
    print(len(vertices))
    X.append(vertices.flatten())

X = np.stack(X)
print(X.shape)

pca = PCA(n_components=2)
pca.fit(X)

	Load mesh from path: /Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-02.ply
19132
	Load mesh from path: /Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-03.ply
18056
	Load mesh from path: /Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-04.ply
17235
	Load mesh from path: /Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-05.ply
17406
	Load mesh from path: /Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-06.ply
17460
	Load mesh from path: /Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-07.ply
17772
	Load mesh from path: /Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-08.ply
18727
	Load mesh from path: /Volumes/GoogleDrive/My Drive/code/my28brains/d

ValueError: all input arrays must have the same shape

## TODO: non-rigid registration of a sphere / the base mesh with given number of vertices. use registered spheres

https://trimsh.org/trimesh.registration.html